# Welcome to your GWFish tutorial

## Disclaimer: no need to install anything, you just need your Google account :-)

Install this branch of GWFish from my GitHub repo, where I modified the original GWfish to take detector-frame masses (so you don't need redshift info) and NO Earth rotation. If you want any modifications, ask !

In [ ]:
! pip install -q git+https://github.com/u-dupletsa/GWFish-1

In [ ]:
! pip install -q lalsuite

In [ ]:
import GWFish.modules as gw
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yaml

## Create injection dataset

In [ ]:
# Function to create your injection set
# mass1, mass2: masses in Msol in detector-frame
# dL: distance in Mpc
# a1, a2: dimensionless spins
# nev: number of events to generate

def create_your_injection_set(mass1, mass2, dL, a1, a2, nev):
    df = {
    'mass_1': mass1*np.ones(nev),
    'mass_2': mass2*np.ones(nev),
    'luminosity_distance': dL*np.ones(nev),
    'dec': np.arcsin(np.random.uniform(-1., 1., nev)),
    'ra': np.random.uniform(0., 2*np.pi, nev),
    'theta_jn': np.arccos(np.random.uniform(-1., 1., nev)),
    'psi': np.random.uniform(0., np.pi, nev),
    'phase': np.random.uniform(0., 2*np.pi, nev),
    'geocent_time': np.random.uniform(2051222400, 2051222400 + 365*24*60*60, nev), # from Jan 1 2035
    'a_1': a1*np.ones(nev),
    'a_2': a2*np.ones(nev)
    }
    df = pd.DataFrame(df)
    return df
# It returns a pandas dataframe with the parameters of the injections, which is what GWfish takes as injection

In [ ]:
np.random.seed(42) # fix so you can obtain the same dataset every time you run the code
# Create your dataset of injections
m1 = 50
m2 = 25
dL = 100
a1 = 0.9
a2 = 0.7
# all the other parameters get automatically randomized
my_dataset_lowq = create_your_injection_set(m1, m2, dL, a1, a2, 50)
my_dataset_extremeq = create_your_injection_set(100, 0.0001, dL, a1, a2, 50)

In [ ]:
Msol = 1.9885e30
c = 299792458.
G = 6.674e-11

def fisco(mass1, mass2):
    M = (mass1 + mass2) * Msol
    return 1 / (np.pi) * c ** 3 / (G * M) / 6 ** 1.5

In [ ]:
# Calculate the ISCO frequency for the injections, so to know when to cut the frequency vector at the higher end
# The frequency vectors starts at 3Hz
freq_max_lowq = fisco(50, 25)
print('The ISCO frequency for low q is:', freq_max_lowq)
freq_max_extremeq = fisco(100, 0.0001)
print('The ISCO frequency for extreme q is:', freq_max_extremeq)

In [ ]:
# Create folder where to store the info about the detector configuration
! mkdir my_yaml_files

In [ ]:
# Create your detector configuration
dict_template = {'ET_HF':{'lat':(40 + 31. / 60 ) * np.pi / 180.,
                    'lon':(9 + 25. / 60) * np.pi / 180.,
                    'opening_angle':np.pi / 3.,
                    'azimuth':70.5674 * np.pi / 180.,
                    'psd_data':'ET_10_HF_psd.txt',
                    'duty_factor':0.85,
                    'detector_class':'earthDelta',
                    'plotrange':'10, 1000, 1e-25, 1e-20',
                    'fmin':3,
                    'fmax':int(freq_max_lowq), ###### CHANGE THIS ACCORDING TO YOUR F_ISCO!!! ######
                    'spacing':'geometric',
                    'df':1/4,
                    'npoints':5000
    }}
with open('my_yaml_files/et_hf_detector_lowq.yaml', 'w') as my_yaml_file:
            yaml.dump(dict_template, my_yaml_file)
# this is the file to pass to GWFish where it will find the detector configuration infos
ConfigDet_lowq = 'my_yaml_files/et_hf_detector_lowq.yaml'

In [ ]:
# Create your detector configuration
dict_template = {'ET_HF':{'lat':(40 + 31. / 60 ) * np.pi / 180.,
                    'lon':(9 + 25. / 60) * np.pi / 180.,
                    'opening_angle':np.pi / 3.,
                    'azimuth':70.5674 * np.pi / 180.,
                    'psd_data':'ET_10_HF_psd.txt',
                    'duty_factor':0.85,
                    'detector_class':'earthDelta',
                    'plotrange':'10, 1000, 1e-25, 1e-20',
                    'fmin':3,
                    'fmax':int(freq_max_extremeq), ###### CHANGE THIS ACCORDING TO YOUR F_ISCO!!! ######
                    'spacing':'geometric',
                    'df':1/4,
                    'npoints':5000
    }}
with open('my_yaml_files/et_hf_detector_extremeq.yaml', 'w') as my_yaml_file:
            yaml.dump(dict_template, my_yaml_file)
# this is the file to pass to GWFish where it will find the detector configuration infos
ConfigDet_extremeq = 'my_yaml_files/et_hf_detector_extremeq.yaml'

In [ ]:
# Create folder where to save gwfish results
! mkdir my_gwfish_results

In [ ]:
data_folder = 'my_gwfish_results'

In [ ]:
waveform = 'TaylorF2'
fisher_parameters = ['mass_1', 'mass_2', 'luminosity_distance', 'dec', 'ra', 'theta_jn', 'psi', 'phase', 'geocent_time',
                    'a_1', 'a_2']

In [ ]:
# Analyze the Fisher matrix for the low q dataset
network = gw.detection.Network(detector_ids = ['ET_HF'], detection_SNR = (0., 8.), config = ConfigDet_lowq)
name_for_output_label_lowq = 'my_injections_lowq'
gw.fishermatrix.analyze_and_save_to_txt(network = network,
                                            parameter_values  = my_dataset_lowq,
                                            fisher_parameters = fisher_parameters, 
                                            sub_network_ids_list = [[0]],
                                            population_name = name_for_output_label_lowq,
                                            waveform_model = waveform,
                                            save_path = data_folder,
                                            save_matrices = False)

In [74]:
# Analyze the Fisher matrix for the extremely low q dataset
network = gw.detection.Network(detector_ids = ['ET_HF'], detection_SNR = (0., 8.), config = ConfigDet_extremeq)
name_for_output_label_extremeq = 'my_injections_extremeq'
gw.fishermatrix.analyze_and_save_to_txt(network = network,
                                            parameter_values  = my_dataset_extremeq,
                                            fisher_parameters = fisher_parameters, 
                                            sub_network_ids_list = [[0]],
                                            population_name = name_for_output_label_extremeq,
                                            waveform_model = waveform,
                                            save_path = data_folder,
                                            save_matrices = False)

100%|██████████| 50/50 [01:28<00:00,  1.77s/it]


In [ ]:
# these are the labels of the columns of the gwfish output
lbs = ['network_SNR', 'mass_1', 'mass_2', 'luminosity_distance', 'dec', 'ra', 'theta_jn', 'psi', 'phase', 'geocent_time', 'a_1', 'a_2',
       'err_mass_1', 'err_mass_2', 'err_luminosity_distance', 'err_dec', 'err_ra', 'err_theta_jn', 'err_psi', 'err_phase',
       'err_geocent_time', 'err_a_1', 'err_a_2', 'err_sky_loc']

In [ ]:
my_output_lowq = pd.read_csv('%s/Errors_ET_HF_%s_SNR8.txt' %(data_folder, name_for_output_label_lowq), names = lbs, sep = ' ', skiprows = 1)
my_output_lowq = my_output_lowq.dropna() #make sure you jus avoid problematic events wuth NaNs

In [ ]:
# Visualize the results
my_output_lowq.head(10)

In [ ]:
my_output_extremeq = pd.read_csv('%s/Errors_ET_HF_%s_SNR8.txt' %(data_folder, name_for_output_label_extremeq), names = lbs, sep = ' ', skiprows = 1)
my_output_extremeq = my_output_extremeq.dropna() #make sure you jus avoid problematic events wuth NaNs

In [ ]:
sub_list_params = ['mass_1', 'mass_2', 'a_1', 'a_2'] # params to plot!

In [ ]:
fig, axs = plt.subplots(2, 2, figsize = (10, 6))
my_bins = np.logspace(-8, 3, 20)
for i, param in enumerate(sub_list_params):
    axs[i//2, i%2].hist(my_output_lowq['err_' + param] / my_output_lowq[param], bins = my_bins, alpha = 0.5, label = 'm1 = 50, m2 = 25, q = 2')
    axs[i//2, i%2].hist(my_output_extremeq['err_' + param] / my_output_extremeq[param], bins = my_bins, alpha = 0.5, label = 'm1 = 100, m2 = 0.0001, q = 1e6')
    axs[i//2, i%2].set_xlabel('relative error on ' + param)
    axs[i//2, i%2].legend()
    axs[i//2, i%2].set_xscale('log')

plt.tight_layout()
plt.show() 